# Generate a Chomo checkpoint file

This notebook generates a Chombo checkpoint file that can be used 
to start GRChombo simulations. 

This example is for simulations with one Scalar Field, note that this is just an example and we do NOT solve the constratints equations (the error is very small though). It corresponds to a slow-rolling inflaton field with small perturbations on the 'phi' values. 

Please note that current script is restricted to seetings with: 
	* Periodic Boundary Conditions 
	* With no ghosts-cells. 

author: Cristian Joana

In [26]:
import numpy as np
import h5py as h5
import os

verbose = 1
overwrite = True
filename = "./test.hdf5"


## Set your GRChombo vars

In [19]:
components = np.array([
    "chi",

    "h11", "h12", "h13", "h22", "h23", "h33",

    "K",

    "A11", "A12", "A13", "A22", "A23", "A33",

    "Theta",

    "Gamma1", "Gamma2", "Gamma3",

    "lapse",

    "shift1", "shift2", "shift3",

    "B1",  "B2", "B3",

    "phi", "Pi",   
])


## Set the grid

In [20]:
params = dict()
all_attrb = dict()
base_attrb = dict()
chombogloba_attrb = dict()
levels_attrb = dict()
boxes = dict()
data_attributes = dict()

# basic params  (MANUAL)
params['N'] = 64 
params['L'] = 1e6
params['dt_multiplier'] = 0.01
params['is_periodic'] = [1, 1, 1]
params['ghosts'] = [0, 0, 0]         # No ghosts possible yet

# set base attibutes (MANUAL)
base_attrb['time'] = 0.0      # float!
base_attrb['iteration'] = 0
base_attrb['max_level'] = 4
base_attrb['num_levels'] = 2  # min = 1 for only level_0
base_attrb['num_components'] = components.size
base_attrb['regrid_interval_0'] = 2
base_attrb['steps_since_regrid_0'] = 0
for comp, name in enumerate(components):
    key = 'component_' + str(comp)
    tt = 'S' + str(len(name))
    base_attrb[key] = np.array(name, dtype=tt)

    
def make_boxes(cords, box_size=8):
    Cx = np.arange(cords[0], cords[3]+1, box_size)
    Cy = np.arange(cords[0], cords[3]+1, box_size)
    Cz = np.arange(cords[0], cords[3]+1, box_size)
    itv = box_size -1
    boxes = []
    for x in Cx:
        for y in Cy:
            for z in Cz:
                box = [x,y,z,x+itv,y+itv,z+itv]
                boxes.append(box)
    return boxes    

# Set Levels & Boxes   (MANUAL)
bend = params['N']-1
boxes["level_0"] = make_boxes([0,0,0,bend,bend,bend],box_size=32) 

bini = 64 - 2*8
bend = bini + 4*8 -1
boxes["level_1"] = make_boxes([bini,bini,bini,bend,bend,bend],box_size=8)

# bini = 128 - 2*8
# bend = bini +4*8-1
# boxes["level_2"] = make_boxes([bini,bini,bini,bend,bend,bend],box_size=8)

# bini = 256 - 2*8
# bend = bini +  4*8-1
# boxes["level_3"] = make_boxes([bini,bini,bini,bend,bend,bend],box_size=8)



In [21]:
# Below are params set automatic. In principle you don't need to change anything.

# def Chombo_global attributes (AUTO)
chombogloba_attrb['testReal'] = 0.0
chombogloba_attrb['SpaceDim'] = 3        # Assuming a 3D grid


# set level attributes and boxes (AUTO)
for il in range(base_attrb['num_levels']):
    levels_attrb['level_{}'.format(il)] = dict()
    ldict = levels_attrb['level_{}'.format(il)]
    ldict['ref_ratio'] = 2
    ldict['dt'] = float(params['L']) / params['N'] * params['dt_multiplier'] / (float(ldict['ref_ratio']) ** il)
    ldict['dx'] = float(params['L']) / params['N'] / (float(ldict['ref_ratio']) ** il)
    ldict['time'] = base_attrb['time']
    ldict['is_periodic_0'] = params['is_periodic'][0]
    ldict['is_periodic_1'] = params['is_periodic'][1]
    ldict['is_periodic_2'] = params['is_periodic'][2]
    ldict['tag_buffer_size'] = 3
    Nlev = int(params['N'] * (int(ldict['ref_ratio']) ** il))
    prob_dom = (0, 0, 0, Nlev - 1, Nlev - 1, Nlev - 1)
    prob_dt = np.dtype([('lo_i', '<i4'), ('lo_j', '<i4'), ('lo_k', '<i4'),
                        ('hi_i', '<i4'), ('hi_j', '<i4'), ('hi_k', '<i4')])
    ldict['prob_domain'] = np.array(prob_dom, dtype=prob_dt)

    prob_dt = np.dtype([('lo_i', '<i4'), ('lo_j', '<i4'), ('lo_k', '<i4'),
                        ('hi_i', '<i4'), ('hi_j', '<i4'), ('hi_k', '<i4')])
    lev_box = np.array([tuple(elm) for elm in boxes["level_{}".format(il)]], dtype=prob_dt)
    boxes["level_{}".format(il)] = lev_box


# set "data attributes" directory in levels, always the same.  (AUTO)
dadt = np.dtype([('intvecti', '<i4'), ('intvectj', '<i4'), ('intvectk', '<i4')])
data_attributes['ghost'] = np.array(tuple(params['ghosts']), dtype=dadt)
data_attributes['outputGhost'] = np.array((0, 0, 0), dtype=dadt)
data_attributes['comps'] = base_attrb['num_components']
data_attributes['objectType'] = np.array('FArrayBox', dtype='S9')


## Set initial values (science problem) 

In [22]:
dic_curv = dict()
mp = 1.0
Mp =  1 / np.sqrt(8.0*np.pi)
phi_ini =  0.5   #   in mp 

# Potential params:      
a = 1e-6   # mass     
b = 1e-3   # lambda

def V(phi, a=a, b=b):     #    V(x) = 0.5 a^2 x^2  + 0.25 b^4 x^4 
    return  0.5* (a * phi)**2 + 0.25 * (b * phi) **4 

def dV_dphi(phi, a=a, b=b, v=0):
    dVdphi =  a**2 * phi +  b**4 * phi**3
    return dVdphi

def get_Pi_init(phi, V=V ,dV = dV_dphi):
    # assuming slowroll
    eta =  - dV(phi) * np.sqrt(3/V(phi) ) * Mp
    return  eta

def epsilon(phi, V=V, dV = dV_dphi):
      return 0.5* Mp**2 * (dV(phi)/V(phi))**2
    

# initial homogeneous values 
Pi_ini =  get_Pi_init(phi_ini)
rho_ini =  0.5*Pi_ini**2 + V(phi_ini)
K_ini = - np.sqrt(24*np.pi* rho_ini)

print(f' Initially: mean rho = {rho_ini} , K = {K_ini}, V = {V(phi_ini)}')

 Initially: mean rho = 3.064113990540577e-13 , K = -4.806545038368574e-06, V = 1.40625e-13


Now we create functions for the inhomogeneous variables. 
They need to be functions of the x,y,z coordinates. 

In [23]:
# parms for generating 'phi'
np.random.seed(124569)
modes = 8
phase = np.random.rand(modes)

# function to generate fluctuations in the SF. 
def _fluc(x, y, z, phase):
    fluc = 0
    for ik in range(modes):
        kk = ik + 1
        LL  = (2*np.pi) / params['L'] * kk    # Assuming L is like R_H.
        RH = np.abs(3/K_ini)
        AmpP2 =  0.5*LL/kk**2
        fluc += AmpP2 * \
                        (np.sin(x *  (LL) + phase[ik]*2*np.pi ) + \
                         np.sin(y *  (LL) +  phase[ik]*2*np.pi ) + \
                         np.sin(z  *  (LL) + phase[ik]*2*np.pi  ))
    return fluc


def _phi(x, y, z):
    fluc = _fluc(x,y,z, phase)
    phi = fluc + phi_ini

    return phi

With the following list, we set up the instructions to build the initial data. 

All GRChombo vars should be included, together with an integer/float or a function:
* If a integer/float is given, this sets up this value everywhere (homogeneous). 
* if a function is given, the function is used to set up its value as a function of x,y,z. 


In [24]:
components_vals = [
    ['chi', 1],
    ['h11', 1], ['h22', 1], ['h33', 1],
    ['h12', 0], ['h13', 0], ['h23', 0],
    ['K', K_ini],
    ['A11', 0], ['A22', 0], ['A33', 0],
    ['A12', 0], ['A13', 0], ['A23', 0],
    ['Theta',0],
    ['Gamma1', 0], ['Gamma2', 0], ['Gamma3', 0],
    ['lapse', 1],
    ['shift1', 0], ['shift2', 0], ['shift3', 0],
    ['B1', 0], ['B2', 0], ['B3', 0],
    ['phi', _phi], ['Pi', Pi_ini]
]
components_vals = np.array(components_vals)

## Create the HDF5 (Automatic)

In [25]:
if overwrite:
    if os.path.exists(filename):
        os.remove(filename)
else:
    if os.path.exists(filename):
        raise Exception(">> The file already exists, and set to not overwrite.")

h5file = h5.File(filename, 'w')  # New hdf5 file I want to create

# base attributes
if verbose > 0: print("Setting base attrs...")
for key in base_attrb.keys():
    h5file.attrs[key] = base_attrb[key]

# group: Chombo_global
if verbose > 0: print("Setting 'chombo global' attrs...")
chg = h5file.create_group('Chombo_global')
for key in chombogloba_attrb.keys():
    chg.attrs[key] = chombogloba_attrb[key]

# group: levels
if verbose > 0: print("Setting levels...")
metadic = dict()
for il in range(base_attrb['num_levels']):
    level_id = 'level_{}'.format(int(il))
    if verbose > 0: print("     creating ", level_id)
    lev = h5file.create_group(level_id)
    level_attrb = levels_attrb[level_id]
    for key in level_attrb.keys():
        if verbose>2: print( "      key-att is", key,  "      value-att is  ", level_attrb[key])
        lev.attrs[key] = level_attrb[key]
    sl = lev.create_group('data_attributes')
    sl.attrs['ghost'] = data_attributes['ghost']
    sl.attrs['outputGhost'] = data_attributes['outputGhost']
    sl.attrs['comps'] = base_attrb['num_components']
    sl.attrs['objectType'] = data_attributes['objectType']

    lev.create_dataset("Processors", data=np.array([0]))
    if verbose > 2: print("     boxes is", boxes)
    boxes_lev = np.array(boxes[level_id])
    lev.create_dataset("boxes", data=boxes_lev)


    Nlev = params['N'] * 2 ** (il)
    dd_lev = params['L'] / Nlev
    boxes_lev = np.array(boxes[level_id].tolist())
    offsets = [0]
    fdset = []  # list containing all box-data (flatten)
    #lev.create_dataset("data:datatype=0", data=np.array([]), chunks=True , maxshape=(None, 1)  )
    lev.create_dataset("data:datatype=0", shape=(0,), chunks=True , maxshape=(None, )  )
    for ib, lev_box in enumerate(boxes_lev):
        if verbose > 2:
            print("  {} box of level {}".format(ib, il))
        X = np.arange(lev_box[0], lev_box[3]+1)
        Y = np.arange(lev_box[1], lev_box[4]+1)
        Z = np.arange(lev_box[2], lev_box[5]+1)
        cord_grid_check = False
        for ic, comp in enumerate(components):
            comp_grid = np.zeros((len(X), len(Y), len(Z)))
            try:
                cid = np.where(components_vals[:, 0] == comp)[0][0]
                eval = components_vals[cid, 1]
            except Exception as e:
                print(" !! component {} not found, values set to zero".format(comp))
                print("   Execption: ", e)
                eval = 0
            if callable(eval):
                # Create coordinate grids
                if not cord_grid_check:
                    cord_grid_check = True
                    x_cord_grid = comp_grid.copy()
                    y_cord_grid = comp_grid.copy()
                    z_cord_grid = comp_grid.copy()
                    # loop over all coords
                    for ix, px in enumerate(X):
                        for iy, py in enumerate(Y):
                            for iz, pz in enumerate(Z):
                                dcnt = 0.5  # cell centering 
                                x_cord_grid[ix, iy, iz] = (px + dcnt) * dd_lev
                                y_cord_grid[ix, iy, iz] = (py + dcnt) * dd_lev
                                z_cord_grid[ix, iy, iz] = (pz + dcnt) * dd_lev
                comp_grid = eval(x_cord_grid, y_cord_grid, z_cord_grid)
                if comp_grid.size != x_cord_grid.size:
                    print("data size does not agree with mesh size")
                    print(" grid size {}, data size {}".format(x_cord_grid.size, comp_grid.size))
                    raise ValueError
            else:
                try:
                    eval = float(eval)
                except ValueError:
                    print("data eval is not a function or digit  --> ", eval)
                    raise
                comp_grid = comp_grid.copy() + eval

            fc = comp_grid.T.flatten()

            levdat = lev["data:datatype=0"]
            levdat.resize((levdat.shape[0] + fc.shape[0]), axis=0)
            levdat[-fc.shape[0]:] = fc


            if il ==0:
                metadic[comp] = [np.mean(fc), np.min(fc), np.max(fc)]
            else:
                metadic[comp] = [metadic[comp][0], \
                                np.min([np.min(fc),metadic[comp][1]]), \
                                np.max([np.max(fc),metadic[comp][2]])]				

            # Cleanining 
            # del  comp_grid
        offsets.extend([len(fdset)])

    offsets = np.array(offsets)
    lev.create_dataset("data:offsets=0", data=offsets)

h5file.close()




################# Print a Summary ######################

if verbose > 0:
    print("")
    print("Summary checkfile:  [average, min, max]")
    for comp in metadic:
        print(comp, "\t\t-->\t\t", metadic[comp])

Setting base attrs...
Setting 'chombo global' attrs...
Setting levels...
     creating  level_0
     creating  level_1

Summary checkfile:  [average, min, max]
chi 		-->		 [1.0, 1.0, 1.0]
h11 		-->		 [1.0, 1.0, 1.0]
h12 		-->		 [0.0, 0.0, 0.0]
h13 		-->		 [0.0, 0.0, 0.0]
h22 		-->		 [1.0, 1.0, 1.0]
h23 		-->		 [0.0, 0.0, 0.0]
h33 		-->		 [1.0, 1.0, 1.0]
K 		-->		 [-4.8065450383685734e-06, -4.806545038368574e-06, -4.806545038368574e-06]
A11 		-->		 [0.0, 0.0, 0.0]
A12 		-->		 [0.0, 0.0, 0.0]
A13 		-->		 [0.0, 0.0, 0.0]
A22 		-->		 [0.0, 0.0, 0.0]
A23 		-->		 [0.0, 0.0, 0.0]
A33 		-->		 [0.0, 0.0, 0.0]
Theta 		-->		 [0.0, 0.0, 0.0]
Gamma1 		-->		 [0.0, 0.0, 0.0]
Gamma2 		-->		 [0.0, 0.0, 0.0]
Gamma3 		-->		 [0.0, 0.0, 0.0]
lapse 		-->		 [1.0, 1.0, 1.0]
shift1 		-->		 [0.0, 0.0, 0.0]
shift2 		-->		 [0.0, 0.0, 0.0]
shift3 		-->		 [0.0, 0.0, 0.0]
B1 		-->		 [0.0, 0.0, 0.0]
B2 		-->		 [0.0, 0.0, 0.0]
B3 		-->		 [0.0, 0.0, 0.0]
phi 		-->		 [0.49999454355393047, 0.4999850096333964, 0.500009236

## utils: other functions

In [1]:
# These set of functions is to set up a Gaussian fluctuation in the scalar curvature (so in 'chi'). 
# It computes the associated ricci_scalar and delta_rho. 

# !!! Needs testing: use at your own risk! 

dic_curv = dict()
HR = 1.
dic_curv['A_gauss'] =   1
dic_curv['S_gauss'] =   5 

def _curv(x, y, z, set_rc=False, rc = False, A=False, B=False):
    """
    x,y,z : These are the cell-centered conformal physical coordinates  ( grid-cords-centered * N_lev/ L )
            usually they are given as 3D arrays. size :(dim=3, Nx_box, Ny_box, Nz_box)
    set_rc: set center at given r
    A:		set amplitude of the Gauss fluc.
    B:		set variance of the Guass fluc. 
    """
    L = params['L']
    vec = np.array([x, y, z])
    if not set_rc :  rc = np.zeros_like(vec) + L/2

    cvec = vec - rc

    if not A: A_gauss = dic_curv['A_gauss']
    if not B: B = dic_curv['S_gauss']

    r2 = cvec[0, :] ** 2 + cvec[1, :] ** 2  + cvec[2, :] ** 2 

    return A_gauss * np.exp(- 0.5 * r2/B**2)


def _drho_th(x, y, z):

    out = 1.0 * _ricci_scalar(x, y, z)/ (16 * np.pi)
    return out


def get_center(x,y,z, cnt):
    crd = np.zeros_like(x)
    center = np.array([crd+cnt[0], crd+cnt[1], crd+cnt[2]])

    return center


def _psi(x, y, z):
    out = np.exp( 0.5 * _curv(x, y, z) )
    return out

def _chi(x, y, z):
    out = _psi(x, y, z)**-4    # Here psi as in Baumgarte book p. 56
    return out

def _ricci_scalar(x, y, z):

    A = dic_curv['A_gauss']
    B = dic_curv['S_gauss']	
    G = _curv(x, y, z) / A
    L = params['L']
    vec = np.array([x, y, z])
    rc = np.zeros_like(vec) + L / 2
    cvec = vec - rc
    r2 = cvec[0, :] ** 2 + cvec[1, :] ** 2  + cvec[2, :] ** 2 


    Omega = (A * np.exp(0.5*A*G - r2/(2*B**2)))/(2*B**2)
    def dderv(cord):
        cc2 = cord**2 
        return (A * cc2* G/(2*B**2) + cc2/B**2) * Omega - Omega

    coord = cvec[0]
    ddx = dderv(coord)
    coord = cvec[1]
    ddy = dderv(coord)
    coord = cvec[2]
    ddz = dderv(coord)	

    out =  - 8 * _psi(x, y, z)**-5 * ( ddx + ddy + ddz)
    return out